# 4.3 - Multiclass Classification With Reuters Dataset

* [4.3.1 - The Boston Housing Prices Dataset](#first-bullet)
* [4.3.2 - Preparing the Data](#second-bullet)
* [4.3.3 - Building Your Model](#third-bullet)
* [4.3.4 - Further Experiments](#fourth-bullet)

The overall goal is to predict the median house prices of homes in a given Boston suburb in the mid 1970's.

Setup and data import:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Get the shape of the data set:

In [4]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


Our targets are median home values in thousands of dollars.  We'll show the first 10.

In [6]:
train_targets[0:9]

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6])